In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import( 
    RFECV,
    SequentialFeatureSelector,
    SelectFromModel
)

from src.wrapped import Wrapped
from src.analyses import * 
from src.train import *

In [2]:
wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [3]:
# dataframe
df = wp.load_data('df_instrumentos_features_selecionadas').drop(columns=['file_name'])

# dataframes por instrumentos
inst_corda = ["cello", "guitar", "violin", "bass", "banjo", "mandolin", "ukulele"]
inst_percusao = ["mallet_percussion", "drums", "cymbals"]
inst_sopro = ["clarinet", "trombone", "flute", "trumpet", "saxophone"]
inst_aerofone = ["accordion", "organ", "piano"] 

df_inst_aerofone = df[df['instrumento'].isin(inst_aerofone)]
df_inst_sopro = df[df['instrumento'].isin(inst_sopro)]
df_inst_corda = df[df['instrumento'].isin(inst_corda)]
df_inst_percusao = df[df['instrumento'].isin(inst_percusao)]

# remover instrumento de corda 
inst_selecionados = np.concatenate((inst_percusao, inst_sopro, inst_aerofone))
new_df = df[df['instrumento'].isin(inst_selecionados)]

# Pre-processamento

- Label Encoder

In [4]:
le = LabelEncoder()

df_inst_aerofone['labels'] = le.fit_transform(df_inst_aerofone.instrumento)
df_inst_sopro['labels'] = le.fit_transform(df_inst_sopro.instrumento)
df_inst_corda['labels'] = le.fit_transform(df_inst_corda.instrumento)
df_inst_percusao['labels'] = le.fit_transform(df_inst_percusao.instrumento)
new_df['labels'] = le.fit_transform(new_df.instrumento)

## Remover Outilers 

- Toda a base menos os instrumentos de corda

In [5]:
train_models(remove_outilers(new_df.drop(columns=['instrumento'])))

Acuracia do modelo GaussianNB() do Fold 0: 0.34576271186440677
Acuracia do modelo GaussianNB() do Fold 1: 0.384180790960452
Acuracia do modelo GaussianNB() do Fold 2: 0.3728813559322034
Acuracia do modelo GaussianNB() do Fold 3: 0.3807909604519774
Acuracia do modelo GaussianNB() do Fold 4: 0.38531073446327685
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.2248587570621469
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.23389830508474577
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.2384180790960452
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.20903954802259886
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.27005649717514124
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.2847457627118644
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.28361581920903955
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.3107344632768362
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.2824858757062147
Acuracia do modelo DecisionTr

{'Naive Bayes': 0.3737853107344633,
 'KNN': 0.23525423728813558,
 'Arvore de Decisão': 0.29355932203389834,
 'Floresta Aleatoria': 0.43299435028248584,
 'HistGradientBoosting': 0.46689265536723157,
 'LIGHTGBM': 0.46892655367231645,
 'MLP': 0.20994350282485877,
 'XGB': 0.4567231638418079,
 'SVC': 0.2711864406779661}

- Instrumento de corda

In [6]:
train_models(remove_outilers(df_inst_corda.drop(columns=['instrumento'])))

Acuracia do modelo GaussianNB() do Fold 0: 0.4105691056910569
Acuracia do modelo GaussianNB() do Fold 1: 0.39227642276422764
Acuracia do modelo GaussianNB() do Fold 2: 0.4715447154471545
Acuracia do modelo GaussianNB() do Fold 3: 0.4623217922606925
Acuracia do modelo GaussianNB() do Fold 4: 0.45213849287169044
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.2926829268292683
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.2784552845528455
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.3130081300813008
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.3014256619144603
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.3095723014256619
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.33739837398373984
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.3516260162601626
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.3780487804878049
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.3380855397148676
Acuracia do modelo DecisionTree

{'Naive Bayes': 0.43777010580696435,
 'KNN': 0.29902886096070735,
 'Arvore de Decisão': 0.34905618200784855,
 'Floresta Aleatoria': 0.5223767655191827,
 'HistGradientBoosting': 0.5329566340469921,
 'LIGHTGBM': 0.5402737072177238,
 'MLP': 0.3022866888546686,
 'XGB': 0.5305101584620735,
 'SVC': 0.3592295464706175}

- Instrumento percusão

In [7]:
train_models(remove_outilers(df_inst_percusao.drop(columns=['instrumento'])))

Acuracia do modelo GaussianNB() do Fold 0: 0.6832740213523132
Acuracia do modelo GaussianNB() do Fold 1: 0.6654804270462633
Acuracia do modelo GaussianNB() do Fold 2: 0.6571428571428571
Acuracia do modelo GaussianNB() do Fold 3: 0.7071428571428572
Acuracia do modelo GaussianNB() do Fold 4: 0.6857142857142857
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.5480427046263345
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.5302491103202847
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5071428571428571
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.5607142857142857
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5571428571428572
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.6120996441281139
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.5836298932384342
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.6214285714285714
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.6571428571428571
Acuracia do modelo DecisionTreeCla

{'Naive Bayes': 0.6797508896797153,
 'KNN': 0.5406583629893238,
 'Arvore de Decisão': 0.625574478901881,
 'Floresta Aleatoria': 0.7353838332486019,
 'HistGradientBoosting': 0.7468327402135231,
 'LIGHTGBM': 0.7439883070665989,
 'MLP': 0.5156863243518047,
 'XGB': 0.7439933909506863,
 'SVC': 0.5706354855109304}

- Instrumento de Sopro

In [8]:
train_models(remove_outilers(df_inst_sopro.drop(columns=['instrumento'])))

Acuracia do modelo GaussianNB() do Fold 0: 0.5379310344827586
Acuracia do modelo GaussianNB() do Fold 1: 0.6448275862068965
Acuracia do modelo GaussianNB() do Fold 2: 0.596551724137931
Acuracia do modelo GaussianNB() do Fold 3: 0.5847750865051903
Acuracia do modelo GaussianNB() do Fold 4: 0.6332179930795848
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.4517241379310345
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4862068965517241
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.47750865051903113
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5363321799307958
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.5344827586206896
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.5379310344827586
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.5551724137931034
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.5467128027681661
Acuracia do modelo DecisionTreeClassifier() do Fo

- Instrumento Aerofones

In [ ]:
train_models(remove_outilers(df_inst_aerofone.drop(columns=['instrumento'])))

# Correlação de -0.5 a 0.5

- Toda a base menos os instrumentos de corda

In [ ]:
df_aux = features_corr(new_df, -0.5, 0.5).drop(columns=['instrumento'])
train_models(df_aux)

- Percusão

In [ ]:
df_aux = features_corr(df_inst_percusao, -0.5, 0.5).drop(columns=['instrumento'])
train_models(df_aux)

- Sopro

In [ ]:
df_aux = features_corr(df_inst_sopro, -0.5, 0.5).drop(columns=['instrumento'])
train_models(df_aux)

- Aerofones

In [ ]:
df_aux = features_corr(df_inst_aerofone, -0.5, 0.5).drop(columns=['instrumento'])
train_models(df_aux)